# Text Mining - Clasificacion textos

### Source: Andrew Task, Udacity


### 1. Carga de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews.txt to reviews.txt


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving reviews.txt to reviews (1).txt


In [ ]:
## Críticas de películas
#g = open('/content/drive/My Drive/Colab Notebooks/AFI/Text Mining/reviews.txt','r') # What we WANT to know!
g = open('/content/reviews.txt','r') 
reviews = g.read().splitlines()
g.close()

## Sentimiento asociado
#g = open('/content/drive/My Drive/Colab Notebooks/AFI/Text Mining/labels.txt','r') # What we WANT to know!
g = open('/content/labels.txt','r') # What we WANT to know!
labels = g.read().upper().splitlines()
g.close()

In [ ]:
len(reviews)

25000

In [ ]:
reviews[1]

'story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers . unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting . even those from the era should be turned off . the cryptic dialogue would make shakespeare seem easy to a third grader . on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond . future stars sally kirkland and frederic forrest can be seen briefly .  '

In [ ]:
len(labels)

25000

In [ ]:
labels[1]

'NEGATIVE'

In [ ]:
def pretty_print_review_and_label(i):
    print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

In [ ]:
print("labels.txt \t : \t reviews.txt\n")
pretty_print_review_and_label(2137)
pretty_print_review_and_label(12816)
pretty_print_review_and_label(6267)
pretty_print_review_and_label(21934)
pretty_print_review_and_label(5297)
pretty_print_review_and_label(4998)

labels.txt 	 : 	 reviews.txt

NEGATIVE	:	this movie is terrible but it has some good effects .  ...
POSITIVE	:	adrian pasdar is excellent is this film . he makes a fascinating woman .  ...
NEGATIVE	:	comment this movie is impossible . is terrible  very improbable  bad interpretat...
POSITIVE	:	excellent episode movie ala pulp fiction .  days   suicides . it doesnt get more...
NEGATIVE	:	if you haven  t seen this  it  s terrible . it is pure trash . i saw this about ...
POSITIVE	:	this schiffer guy is a real genius  the movie is of excellent quality and both e...


### 2. Análisis cuantitativo términos: ¿Qué términos aparecen en los comentarios positivos, cuales en los negativos y cuales aparecen en ambos?

In [ ]:
from collections import Counter
import numpy as np

Vamos a utilizar la estructura Counter de python para ver cuantas veces aparece cada palabra en las críticas. Debajo tienes un ejemplo de como utilizar un contador.

https://docs.python.org/2/library/collections.html#collections.Counter

In [ ]:
ex_count = Counter()
ex_count['coche'] = 3
ex_count['palabra2'] = 5
ex_count.most_common()

# objeto "Counter()" de la librería colección sirve para crear lista de tuplas
# en este caso lo vamos a utilizar para crear una lista de tuplas con key=palabra y value=frecuencia palabra

[('palabra2', 5), ('coche', 3)]

In [ ]:
# Un contandor para cada tipo de review y uno total
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()

In [ ]:
# Crea un bucle que, para cada crítica, recorra sus palabras una a una e incremente en 1 el número de aparaciones.
# Aumenta el contador siempre en total_counts y en positive_counts O en negative_counts dependiendo de si es una crítica
# positiva o negativa

for i in range(len(reviews)):
  if(labels[i] == "POSITIVE"):
    for word in reviews[i].split(" "):
      positive_counts[word] += 1
      total_counts[word] +=1
  else:
    for word in reviews[i].split(" "):
      negative_counts[word] += 1
      total_counts[word] +=1

# recorremos todas las reviews, clasificamos negativas y positivas
# para cada tipo recorremos las palabras de cada crítica y creamos objeto "Counter()" (lista de tuplas) 
# con key=palabra y value=frecuencia palabra

In [ ]:
ex_count = Counter()

In [ ]:
ex_count['coche'] += 3
ex_count['coche'] += 1
ex_count['coche'] += 7
ex_count['palabra2'] = 5
ex_count.most_common()

# objeto "Counter()" de la librería colección sirve para crear lista de tuplas
# en este caso lo vamos a utilizar para crear una lista de tuplas con key=palabra y value=frecuencia palabra

[('coche', 11), ('palabra2', 5)]

In [ ]:
positive_counts.most_common(10)

[('', 550468),
 ('the', 173324),
 ('.', 159654),
 ('and', 89722),
 ('a', 83688),
 ('of', 76855),
 ('to', 66746),
 ('is', 57245),
 ('in', 50215),
 ('br', 49235)]

In [ ]:
negative_counts.most_common(10)

# más comunes son "stopwords"

[('', 561462),
 ('.', 167538),
 ('the', 163389),
 ('a', 79321),
 ('and', 74385),
 ('of', 69009),
 ('to', 68974),
 ('br', 52637),
 ('is', 50083),
 ('it', 48327)]

In [ ]:
total_counts.most_common(10)

# más comunes son "stopwords"

[('', 1111930),
 ('the', 336713),
 ('.', 327192),
 ('and', 164107),
 ('a', 163009),
 ('of', 145864),
 ('to', 135720),
 ('is', 107328),
 ('br', 101872),
 ('it', 96352)]

El resultado del conteo muestra que las stopwords están presentes tanto en críticas positivas como en críticas negativas y pueden añadir ruido a la hora crear un modelo de clasificación. ¿Cómo podemos sacar aquellas palabras que son un indicador claro de que se trata de una crítica positiva o negativa? 

### 3. Análisis cuantitativo términos: Ratios

Vamos a calcular los ratios de aparición de los términos de la siguiente manera: ratio = positive_counts/float(negative_counts + 1).

**Nota**: vamos a trabajar unicamente con aquellos términos que **en total** aparecen 101 o más veces.

*   ¿Por qué ese +1 en el denominador?
*   A bote pronto, ¿cómo interpretaríamos los resultados? ¿En qué rango se van a mover los ratios?



In [ ]:
pos_neg_ratios = Counter()

# Calcula el ratio para los término más comunes
for term, cnt in list(total_counts.most_common()):
  if(cnt > 100):
    pos_neg_ratios[term] = positive_counts[term] / float(negative_counts[term]+1)

# en la lista "total_counts" anterior (lista de tuplas con todas las palabras del conjunto de textos = vocabulario)
# recorro la lista de tuplas comprobando condición de que su frecuencia sea mayor de 101
# en caso de cumplir condición, sacar valor que queremos calcular

# "float()" lo ponemos para asegurarnos de que nos devuelve los decimales de la división

In [ ]:
pos_neg_ratios.most_common(15)

# palabras con valores mayores = palabras "positivas"

[('edie', 109.0),
 ('paulie', 59.0),
 ('felix', 23.4),
 ('polanski', 16.833333333333332),
 ('matthau', 16.555555555555557),
 ('victoria', 14.6),
 ('mildred', 13.5),
 ('gandhi', 12.666666666666666),
 ('flawless', 11.6),
 ('superbly', 9.583333333333334),
 ('perfection', 8.666666666666666),
 ('astaire', 8.5),
 ('captures', 7.68),
 ('voight', 7.615384615384615),
 ('wonderfully', 7.552631578947368)]

In [ ]:
## Algunos ejemplos que nos ayudan a interpretar los resultados
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

# palabras en ambos tipos de reviews ("stopwords" o palabras genéricas sobre el tema) / valor = 1
# palabras en reviews positivas / valor >>> 1
# palabras en reviews negativas / valor = 0

# PROBLEMAS DE ESCALADO:
# el problema con esta valoración de las palabras es que el espacio para las palabras negativas es muy pequeño [0,1) en
# comparación con el de las palabras positivas (1,infinito)

# si le damos los valores así a un modelo, es muy probable que se vea influenciado por el mayor orden de los valores 
# positivos y puede que no funcione correctamente

# Lo corregimos aplicando logaritmos:
#  - valores (0,1) = valor negativo
#  - valor (1) = 0
#  - valores >>> 1 = valor (0,infinito)  pero va a reducir logaritmicamente el orden de los valores anteriores
# conseguimos igualar/normalizar los valores

Pos-to-neg ratio for 'the' = 1.0607993145235326
Pos-to-neg ratio for 'amazing' = 4.022813688212928
Pos-to-neg ratio for 'terrible' = 0.17744252873563218


¿Qué problema tiene esta definición de ratio? ¿Cómo lo solucionamos?

### 4. Análisis cuantitativo términos: Logaritmos

Al aplicar logaritmos a los valores calculados en el apartado anterior hacemos que los valores por debajo de 1 pasen a ser negativos (y con valor absoluto más alto cuanto más cercanos a 0 sean) y además conseguimos que dos términos con frecuencias relativas parecidas pero en críticas de signo distinto tomen valores con valor absoluto parecido y signo contrario.

In [ ]:
# Calcula el logaritmo de los ratios para todos los términos
for term, ratio in list(pos_neg_ratios.most_common()):
  pos_neg_ratios[term] = np.log(ratio)

# recorremos lista de valores calculados anteriormente y sustituimos el valor por el logaritmo del valor

In [ ]:
pos_neg_ratios.most_common(15)

[('edie', 4.6913478822291435),
 ('paulie', 4.07753744390572),
 ('felix', 3.152736022363656),
 ('polanski', 2.8233610476132043),
 ('matthau', 2.80672172860924),
 ('victoria', 2.681021528714291),
 ('mildred', 2.6026896854443837),
 ('gandhi', 2.538973871058276),
 ('flawless', 2.451005098112319),
 ('superbly', 2.26002547857525),
 ('perfection', 2.159484249353372),
 ('astaire', 2.1400661634962708),
 ('captures', 2.038619547159581),
 ('voight', 2.030170492673053),
 ('wonderfully', 2.0218960560332353)]

In [ ]:
## Algunos ejemplos que nos ayudan a interpretar los resultados
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'terrible' = {}".format(pos_neg_ratios["terrible"]))

# palabras en ambos tipos de reviews ("stopwords" o palabras genéricas sobre el tema) / valor = 0
# palabras en reviews positivas / valor > 0
# palabras en reviews negativas / valor < 0

Pos-to-neg ratio for 'the' = 0.05902269426102881
Pos-to-neg ratio for 'amazing' = 1.3919815802404802
Pos-to-neg ratio for 'terrible' = -1.7291085042663878


In [ ]:
pos_neg_ratios.most_common()[:-31:-1]

[('boll', -4.969813299576001),
 ('uwe', -4.624972813284271),
 ('seagal', -3.644143560272545),
 ('unwatchable', -3.258096538021482),
 ('stinker', -3.2088254890146994),
 ('mst', -2.9502698994772336),
 ('incoherent', -2.9368917735310576),
 ('unfunny', -2.6922395950755678),
 ('waste', -2.6193845640165536),
 ('blah', -2.5704288232261625),
 ('horrid', -2.4849066497880004),
 ('pointless', -2.4553061800117097),
 ('atrocious', -2.4259083090260445),
 ('redeeming', -2.3682390632154826),
 ('prom', -2.3608540011180215),
 ('drivel', -2.3470368555648795),
 ('lousy', -2.307572634505085),
 ('worst', -2.286987896180378),
 ('laughable', -2.264363880173848),
 ('awful', -2.227194247027435),
 ('poorly', -2.2207550747464135),
 ('wasting', -2.204604684633842),
 ('remotely', -2.1972245773362196),
 ('existent', -2.0794415416798357),
 ('boredom', -1.995100393246085),
 ('miserably', -1.9924301646902063),
 ('sucks', -1.987068221548821),
 ('uninspired', -1.9832976811269336),
 ('lame', -1.981767458946166),
 ('insult

### 5. Modelo de clasificación basado en bag of words

Vamos a aplicar la técnica de bag of words paso a paso a cada una de las críticas con el objetivo de convertirlas en vectores numéricos que sirvan de features de un nuestro modelo.

#### Primer paso: construir el conjunto de palabras de nuestros vocabulario.

In [ ]:
vocab = set(total_counts.keys())

# creamos vocabulario con todas las palabras de la lista que calculamos "total_counts"

In [ ]:
vocab_size = len(vocab)
vocab_size

74074

#### Segundo paso: construímos un vector del tamaño de nuestro vocabulario. Para ganar tiempo lo creamos como un vector entero de 0s usando la función de numpy zeros()

In [ ]:
import numpy as np
zeros = np.zeros(vocab_size)

In [ ]:
zeros

array([0., 0., 0., ..., 0., 0., 0.])

#### Tercer paso: asignar a cada palabra un índice del vector y crear una tabla maestra que guarde esta relación y nos permita crear fácilmente nuestros vectores.

In [ ]:
# Creamos un diccionario que tiene como key la palabra y como valor el índice asociado
word2index = {}
for i,word in enumerate(vocab):
    word2index[word] = i
    
word2index

# función que crea un diccionario con las palabras del vocabulario de las reviewz y asigna a cada palabra su índice en el 
# vocabulario de palabras, cuando calculemos los vectores las columnas asociadas a cada palabra coincidirán con estos índices

{'': 0,
 'klown': 1,
 'treck': 2,
 'polson': 3,
 'items': 4,
 'remedios': 5,
 'culpability': 6,
 'hurt': 7,
 'barbs': 8,
 'camped': 9,
 'sudser': 10,
 'olivier': 11,
 'fancies': 12,
 'jaques': 13,
 'beaute': 14,
 'eradicate': 15,
 'hoyo': 16,
 'dickinsons': 17,
 'twilight': 18,
 'beeline': 19,
 'feuds': 20,
 'ghulam': 21,
 'heidi': 22,
 'runaways': 23,
 'carlas': 24,
 'fulfilled': 25,
 'pancreatitis': 26,
 'janikowski': 27,
 'skanky': 28,
 'minbari': 29,
 'vail': 30,
 'cannes': 31,
 'whistlestop': 32,
 'douses': 33,
 'crevice': 34,
 'rodder': 35,
 'besxt': 36,
 'mansion': 37,
 'menace': 38,
 'counteract': 39,
 'homoeroticism': 40,
 'captioning': 41,
 'declares': 42,
 'paglia': 43,
 'mirroed': 44,
 'reginald': 45,
 'itelf': 46,
 'crystin': 47,
 'trivialia': 48,
 'thickly': 49,
 'dasilva': 50,
 'slept': 51,
 'mancini': 52,
 'jungian': 53,
 'buckaroos': 54,
 'semite': 55,
 'burglarize': 56,
 'metroid': 57,
 'meyers': 58,
 'sinnister': 59,
 'bill': 60,
 'appalingly': 61,
 'assimilation': 6

#### Cuarto paso: crear la función que dada una crítica devuelve un vector contando la frecuencia de las palabras utilizadas.

In [ ]:
## Rellena la función que para crítica devuelve el vector asociado aplicando bag of words
def bag_of_words(review):
  v = np.zeros(vocab_size)
  for word in review.split(" "):
    v[word2index[word]] += 1
  return v

# definimos función "bag_of_words()" que saca vector para cada review con la 
# frecuencia de las palabras que aparecen en esa review

In [ ]:
 reviews[0]

In [ ]:
bag_of_words(reviews[0])

array([18.,  0.,  0., ...,  0.,  0.,  0.])

In [ ]:
bag_of_words(reviews[0])[:100]

array([18.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [ ]:
list(word2index)[0]

''

In [ ]:
word2index['']

# palabra con indice=0 del vocabulario es ""

0

In [ ]:
bag_of_words(reviews[0])[0]

# de esta forma estamos revisando para la "review_0" cuantas veces aparece (18 veces) la palabra con índice=0 de nuestro 
# vocabulario
# palabra con indice=0 del vocabulario es ""

18.0

In [ ]:
word2index["bromwell"]

43671

In [ ]:
bag_of_words(reviews[0])[12891]

0.0

In [ ]:
bag_of_words(reviews[0]).shape

(74074,)

#### Quinto paso: el target de nuestro modelo serán 1s y 0s. Vamos a crear una función que convierta los cadenas POSITIVE y NEGATIVE a 1 y 0. 

In [ ]:
## Rellena la función para que dada la etiqueta en forma de cadena devuelve el entero asociado
def target_numerico(label):
  if label == "POSITIVE":
    return 1
  else:
    return 0

In [ ]:
target_numerico(labels[0])

1

#### Sexto paso: dividimos los datos que tenemos en train y test (esta vez lo hacemos a ojo).

In [ ]:
training_size = 5000
test_size = 10000
training_rev = reviews[:training_size]
training_lab = labels[:training_size]
test_rev = reviews[-test_size:]
test_lab = labels[-test_size:]
print(len(training_lab))
print(len(test_lab))

# dividimos dataset en "train" y "test" para entrenar el modelo y luego comprobar su rendimiento

5000
10000


In [ ]:
print(training_rev[training_size-1])
print(reviews[training_size-1])

who in their right mind does anything so stupid as this movie   br    br   accidental killing of a security guard . . . characters that are so two dimensional that a two year old could have painted drawn them . . . and better . . .  br    br   a red toolbox of death  please . . . .  br    br   hypothermic weak thugs . . .  br    br   acting from hell . . .  br    br   stylistically this movie shifts between teen comedy  thriller  voyeurism and . . . female . . .  uhm  rambo   br    br   unbelievable and it  s an insult to any thinking person . do not watch  walk away it  s more horrible than you may imagine . . .  br    br   and on top of it all it  s trying to be hip by being overly graphic in it  s violence . . .  br    br   mrs montford shoot  em up was fun and funny  this is just pathetic and terrible . good luck next time .     
who in their right mind does anything so stupid as this movie   br    br   accidental killing of a security guard . . . characters that are so two dimensi

In [ ]:
print(test_rev[0])
print(reviews[-test_size])

this documentary  now available free on video . google . com  is a fantastic demonstration of the power of ordinary people to overcome injustice . everyone must see this .  br    br   chavez was elected in a landslide vote in     . his platform was to divert the fantastic oil wealth from the    middle class to the    poor . he banned foreign drift net fishing in venezuelan waters . he sent       cuban doctors to the slums to treat the sick for free . he wiped out illiteracy and set up new free universities .  br    br   but it was his    tax on oil company profits that got him in trouble with the bush administration . in      while irish film makers kim bartley and donnacha o  briain were interviewing chavez inside the presidential palace about his social programs  a cia backed coup was launched . with the cameras rolling  chavez was captured and flown out of the country . it was announced on national tv that he had  resigned  .  br    br   but the poor of venezuela didn  t believe the

#### Séptimo paso: calculamos las matrices de entrenamiento y de test.

In [ ]:
X_train = np.empty((len(training_rev), vocab_size))
print(X_train.shape)
### Rellena X aquí
for i in range(len(training_rev)):
  X_train[i] = bag_of_words(training_rev[i])

X_train[:3]

# creamos matriz de "train" añadiendo los vectores de las reviews del conjunto de entrenamiento definido anteriormente

(5000, 74074)


array([[18.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [78.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
X_test = np.empty((len(test_rev), vocab_size))
print(X_test.shape)
# Rellena X_test aquí
for i in range(len(test_rev)):
  X_test[i] = bag_of_words(test_rev[i])

X_test[:3]

# creamos matriz de "test" añadiendo los vectores de las reviews del conjunto de entrenamiento definido anteriormente

(10000, 74074)


array([[53.,  0.,  0., ...,  0.,  0.,  0.],
       [72.,  0.,  0., ...,  0.,  0.,  0.],
       [28.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
y_train = np.empty((len(training_lab),))
print(y_train.shape)
# Rellena y aquí
for i in range(len(training_lab)):
  y_train[i] = target_numerico(training_lab[i])

y_train

(5000,)


array([1., 0., 1., ..., 0., 1., 0.])

In [ ]:
y_test = np.empty((len(test_lab),))
print(y_test.shape)
# Rellena y_test aquí
for i in range(len(test_lab)):
  y_test[i] = target_numerico(test_lab[i])

y_test

(10000,)


array([1., 0., 1., ..., 0., 1., 0.])

#### Octavo paso: entrenamos el modelo

In [ ]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
model = linear_model.LogisticRegression()
model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### Noveno paso: aplicamos el predict sobre el conjunto de test y vemos qué tal funciona.

In [ ]:
predictions = model.predict(X_test)

In [ ]:
model.score(X_train,y_train)

0.9984

In [ ]:
model.score(X_test,y_test)

0.8397

In [ ]:
print(confusion_matrix(y_test, predictions))

[[4311  689]
 [ 914 4086]]


In [ ]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.83      0.86      0.84      5000
         1.0       0.86      0.82      0.84      5000

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



#### Decimo paso: preparamos el código para probar el modelo con cadenas nuevas.

In [ ]:
# Rellena la función para, dada una crítica, aplicar el modelo que hemos entrenado
# e imprimir POSITIVE/NEGATIVE
def sentiment_analysis(review):
  vector_input = np.empty((1, vocab_size))
  vector_input[0] = bag_of_words(review)
  pred = model.predict(vector_input)
  if pred == 1:
    print("POSITIVE")
  else:
    print("NEGATIVE")

In [ ]:
sentiment_analysis('movie bad')

NEGATIVE


In [ ]:
sentiment_analysis('not horrible')

NEGATIVE


In [ ]:
sentiment_analysis('España good')

KeyError: ignored

¿Qué hacemos con el error que se obtiene al meter una palabra que no está en el vocabulario? Solucionar este error es parte de la práctica de la asignatura.

In [ ]:
# al incluir una palabra que no ha visto en train, el modelo se rompe y devuelve un error
# resolver el problema para que no de error

In [ ]:
sentiment_analysis('Cool')

KeyError: ignored

Pero si Cool si es una palabra inglesa. ¿Qué ocurre? ¿Cómo lo solucionamos? Esto también es parte de la práctica de la asignatura. 

In [ ]:
# retocar fórmulas/funciones definidas para que no se rompa el modelo
# resolver el problema para que no de error

#### SOLUCIÓN DE ERRORES: Definición de una nueva función para la aplicación del modelo

In [ ]:
# IDEA (alguna palabra de la nueva review no está o no coincide exactamente con las palabras del diccionario)

# FUNCIÓN DE APLICACIÓN DEL MODELO:
# Revisar la función de aplicación del modelo para que sea capaz de analizar las palabras de la review nueva.
# Comprobar casos:
#    a) palabra pertenece a vocabulario 
#    b) palabra pertenece al vocabulario pero con variación de mayúscula/minúscula inicial 
#        (informar al usuario de que se transforma la palabra e incluir transformación en el análisis)
#    c) palabra no pertenece al vocabulario
#        (informar al usuario de que se descarta la palabra y no se tendrá en cuenta para el análisis)


In [ ]:
# Rellena la función para, dada una crítica, aplicar el modelo que hemos entrenado e imprimir POSITIVE/NEGATIVE

def rev_sentiment_analysis(review):
  vector_input = np.empty((1, vocab_size))
  review_vocab = []
  review_transfvocab = []
  review_notvocab = []

  # Análisis previo de las palabras de la nueva review
  for word in review.split(" "):
    if word in vocab:
      review_vocab.append(word)
      continue
    else: 
      if word.lower() in vocab:
        review_transfvocab.append(word.lower())
        continue
      if word.capitalize() in vocab:
        review_transfvocab.append(word.capitalize())
        continue
      else:
        review_notvocab.append(word)

  review = " ".join(list(review_vocab + review_transfvocab))
  print(' - Palabras transformadas (mayúsculas/minúsculas): ', review_transfvocab)
  print(' - Palabras descartadas (no contenidas en vocabulario de "train"): ', review_notvocab)
  
  # Comprobación y aplicación del modelo
  if len(review) == 0:
    print('-'*100)
    print('No se puede realizar el análisis de sentimiento de la review introducida.')
    print('Ninguna de las palabras incluidas en la review está contenida en el vocabulario de "train" del modelo.')
  else:
    print(' - Palabras consideradas para el análisis de sentimiento: ', review.split(" "))
    print('-'*100)
    vector_input[0] = bag_of_words(review)
    pred = model.predict(vector_input)
    if pred == 1:
      print("POSITIVE")
    else:
      print("NEGATIVE")


In [ ]:
rev_sentiment_analysis('Polanski España good film , polanski')

 - Palabras transformadas (mayúsculas/minúsculas):  ['polanski']
 - Palabras descartadas (no contenidas en vocabulario de "train"):  ['España', ',']
 - Palabras consideradas para el análisis de sentimiento:  ['good', 'film', 'polanski', 'polanski']
----------------------------------------------------------------------------------------------------
POSITIVE


In [ ]:
rev_sentiment_analysis(',')

 - Palabras transformadas (mayúsculas/minúsculas):  []
 - Palabras descartadas (no contenidas en vocabulario de "train"):  [',']
----------------------------------------------------------------------------------------------------
No se puede realizar el análisis de sentimiento de la review introducida.
Ninguna de las palabras incluidas en la review está contenida en el vocabulario de "train" del modelo.


In [ ]:
rev_sentiment_analysis('movie bad')

 - Palabras transformadas (mayúsculas/minúsculas):  []
 - Palabras descartadas (no contenidas en vocabulario de "train"):  []
 - Palabras consideradas para el análisis de sentimiento:  ['movie', 'bad']
----------------------------------------------------------------------------------------------------
NEGATIVE


In [ ]:
rev_sentiment_analysis('not horrible')

 - Palabras transformadas (mayúsculas/minúsculas):  []
 - Palabras descartadas (no contenidas en vocabulario de "train"):  []
 - Palabras consideradas para el análisis de sentimiento:  ['not', 'horrible']
----------------------------------------------------------------------------------------------------
NEGATIVE


In [ ]:
rev_sentiment_analysis('España good')

 - Palabras transformadas (mayúsculas/minúsculas):  []
 - Palabras descartadas (no contenidas en vocabulario de "train"):  ['España']
 - Palabras consideradas para el análisis de sentimiento:  ['good']
----------------------------------------------------------------------------------------------------
POSITIVE


In [ ]:
rev_sentiment_analysis('Cool')

 - Palabras transformadas (mayúsculas/minúsculas):  ['cool']
 - Palabras descartadas (no contenidas en vocabulario de "train"):  []
 - Palabras consideradas para el análisis de sentimiento:  ['cool']
----------------------------------------------------------------------------------------------------
POSITIVE
